In [1]:
pip install tensorflowjs


In [3]:
import pandas as pd
import requests
import os
from IPython.display import display, HTML

os.chdir("/content/drive/MyDrive/Capstone - Bangkit")

In [4]:
!ls

 ConvertToChunk.ipynb   dataset		     model.h5	 'recomendation system'   tes.json
 data		       'Image Model.ipynb'   my_data.db  'step - 1.ipynb'	  tfjs_model


In [12]:
!ls models

model.h5


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')
model.summary()

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='dense_1/Softmax' --saved_model_tags=serve models/model.h5 tfjs_model/

# Preparation

In [ ]:
df = pd.read_json("./data/dataset.json")

In [ ]:
# Fungsi untuk menampilkan gambar dalam tabel
def display_image(img_url):
    return f'<img src="{img_url}" style="max-height:300px; max-width:300px;">'

def display_table(df_, range = False, void = True):
  if range:
    df_ = df_.loc[range][['title', 'image_real']]
  else:
    df_ = df_[['title', 'image_real']]
  # Menambahkan kolom gambar ke dalam tabel
  df_['image_display'] = df_['image_real'].apply(display_image)

  # Menampilkan tabel
  display(HTML(df_[['title', 'image_display']].to_html(escape=False)))
  if void == False:
    return df_

def download_images(dataframe, target_directory):
    """
    Fungsi untuk mengunduh semua gambar dari kolom 'image_real' dalam DataFrame.

    Parameters:
    - dataframe: DataFrame yang berisi kolom 'image_real'.
    - target_directory: Direktori tempat gambar akan disimpan.
    """
    # Pastikan target_directory ada, jika tidak, buat direktori tersebut
    os.makedirs(target_directory, exist_ok=True)

    # Iterasi melalui setiap baris dalam DataFrame
    for index, row in dataframe.iterrows():
        image_url = row['image_real']
        image_name = f"image_{index}.jpg"  # Nama file lokal

        # Mengunduh gambar
        response = requests.get(image_url)
        if response.status_code == 200:
            # Menyimpan gambar ke dalam direktori target
            image_path = os.path.join(target_directory, image_name)
            with open(image_path, 'wb') as f:
                f.write(response.content)
            print(f"Gambar {image_name} berhasil diunduh.")
        else:
            print(f"Gambar {image_name} gagal diunduh. Status code: {response.status_code}")

In [ ]:
df.columns

Index(['link', 'image', 'title', 'id_product', 'image_real', 'product_name',
       'image_display'],
      dtype='object')

In [ ]:
filename = df.product_name.unique()

In [ ]:
filename

array(['Nikon D850', 'Samsung Galaxy S21', 'Google Pixel 5',
       'MacBook Air', 'OXIMUS ZDP1118'], dtype=object)

In [ ]:
df_1 = df[df['product_name'] == filename[0]][:100]
df_2 = df[df['product_name'] == filename[1]][:100]
df_3 = df[df['product_name'] == filename[2]][:100]
df_4 = df[df['product_name'] == filename[3]][:100]
df_5 = df[df['product_name'] == filename[4]][:100]

In [ ]:
df.product_name.value_counts()

MacBook Air           338
OXIMUS ZDP1118        262
Samsung Galaxy S21    235
Google Pixel 5        111
Nikon D850            101
Name: product_name, dtype: int64

In [ ]:
df_2.shape

(100, 6)

In [ ]:
for filename_ in filename:
  download_images(df[df['product_name'] == filename_][:100], filename_)

In [ ]:
df_12 = display_table(df_4, False, False)

# Modeling

In [4]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Layer konvolusi dan pooling pertama
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Layer konvolusi dan pooling kedua
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Layer konvolusi dan pooling ketiga
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer untuk mengubah matriks menjadi vektor
    model.add(layers.Flatten())

    # Dense layer dengan 512 neuron
    model.add(layers.Dense(512, activation='relu'))

    # Output layer dengan num_classes neuron (sesuai jumlah kategori)
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Dimensi input gambar (disesuaikan dengan gambar Anda)
input_shape = (150, 150, 3)

# Jumlah kategori (sesuaikan dengan jumlah kategori Anda)
num_classes = 5

# Membuat model
model = create_cnn_model(input_shape, num_classes)

# Membuat objek ImageDataGenerator untuk augmentasi gambar
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data menjadi data pelatihan dan validasi
)

# Mendefinisikan direktori dataset
dataset_dir = 'dataset'

# Membuat generator data pelatihan
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Data pelatihan
)

# Membuat generator data validasi
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Data validasi
)

Found 400 images belonging to 5 classes.
Found 99 images belonging to 5 classes.


In [6]:
# Menentukan jumlah epoch
epochs = 10

# Pelatihan model
with tf.device("/GPU:0"):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=epochs
    )


Epoch 1/10
12/12 [==============================] - 143s 11s/step - loss: 1.9822 - accuracy: 0.2853 - val_loss: 1.4074 - val_accuracy: 0.4062
Epoch 2/10
12/12 [==============================] - 3s 272ms/step - loss: 1.1211 - accuracy: 0.5951 - val_loss: 1.0101 - val_accuracy: 0.5729
Epoch 3/10
12/12 [==============================] - 3s 276ms/step - loss: 0.7786 - accuracy: 0.6848 - val_loss: 0.6987 - val_accuracy: 0.7292
Epoch 4/10
12/12 [==============================] - 4s 360ms/step - loss: 0.6574 - accuracy: 0.7473 - val_loss: 0.6890 - val_accuracy: 0.7604
Epoch 5/10
12/12 [==============================] - 3s 279ms/step - loss: 0.4885 - accuracy: 0.8342 - val_loss: 0.7226 - val_accuracy: 0.8125
Epoch 6/10
12/12 [==============================] - 3s 268ms/step - loss: 0.4739 - accuracy: 0.8152 - val_loss: 0.6969 - val_accuracy: 0.7396
Epoch 7/10
12/12 [==============================] - 5s 433ms/step - loss: 0.4428 - accuracy: 0.8370 - val_loss: 0.6156 - val_accuracy: 0.8229
Epoch 

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = 'mac.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Menambahkan dimensi batch
img_array /= 255.0  # Normalisasi nilai piksel ke rentang [0, 1]

prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)
print(f'Predicted class: {predicted_class}')
predicted_label = [label for label, index in class_labels.items() if index == predicted_class][0]
print(f'Predicted label: {predicted_label}')


1/1 [==============================] - 0s 45ms/step
Predicted class: 0
Predicted label: Google Pixel 5


In [ ]:
predicted_label = [label for label, index in class_labels.items() if index == predicted_class][0]
print(f'Predicted label: {predicted_label}')


Predicted label: Nikon D850


In [ ]:
train_generator.class_indices

{'Google Pixel 5': 0,
 'MacBook Air': 1,
 'Nikon D850': 2,
 'OXIMUS ZDP1118': 3,
 'Samsung Galaxy S21': 4}

In [ ]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
